In [1]:
import os
from  dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)


True

In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import Chroma
from langchain.indexes import VectorstoreIndexCreator

In [6]:
from langchain.document_loaders import CSVLoader
file = 'files/products_min.csv'
loader = CSVLoader(file_path=file)
docs = loader.load()

In [11]:
index = VectorstoreIndexCreator(
    vectorstore_cls=Chroma
).from_loaders([loader])

In [13]:
llm = ChatOpenAI(temperature = 0.0, model='gpt-3.5-turbo')
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [15]:
data = loader.load()

In [19]:
print(data[10].page_content)

: 10
ProductID: 10000911
ProductName: Kenneth Cole Women Navy Blue Solid Backpack
ProductBrand: Kenneth Cole
Gender: Women
Price (INR): 2463
NumImages: 5
Description: Navy Blue backpackNon-Padded haul loop1 main compartment with zip closurePadded backZip PocketPadded shoulder strap: PaddedWater-resistance: No
PrimaryColor: Blue


In [21]:
print(data[11].page_content)

: 11
ProductID: 10000245
ProductName: Parx Men Green Printed Polo Collar T-shirt
ProductBrand: Parx
Gender: Men
Price (INR): 629
NumImages: 5
Description: Green printed T-shirt, has a polo collar, and short sleeves
PrimaryColor: Green


In [33]:
examples = [
    {
        "query":"IS Kenneth Cole Women Navy Blue Solid Backpack PaddedWater-resistance?",
        "answer":"no"
    },
    {
        "query":"Which product in the Parx brand has a polo collar",
        "answer":"Parx Men Green Printed Polo Collar T-shirt"
    }
]

In [23]:
from langchain.evaluation.qa import QAGenerateChain

In [24]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI(model='gpt-3.5-turbo'))

In [25]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

C:\Users\iss211\llm\llm-env\Lib\site-packages\langchain\chains\llm.py:306: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [29]:
new_examples[0]

{'qa_pairs': {'query': 'What is the product name and brand mentioned in the document?',
  'answer': 'The product mentioned in the document is the "DKNY Unisex Black & Grey Printed Medium Trolley Bag" and the brand is "DKNY".'}}

In [34]:
for example in new_examples:
    examples.append(example['qa_pairs'])

In [35]:
examples

[{'query': 'IS Kenneth Cole Women Navy Blue Solid Backpack PaddedWater-resistance?',
  'answer': 'no'},
 {'query': 'Which product in the Parx brand has a polo collar',
  'answer': 'Parx Men Green Printed Polo Collar T-shirt'},
 {'query': 'What is the product name and brand mentioned in the document?',
  'answer': 'The product mentioned in the document is the "DKNY Unisex Black & Grey Printed Medium Trolley Bag" and the brand is "DKNY".'},
 {'query': 'What is the primary color of the EthnoVogue Women Beige & Grey Made to Measure Custom Made Kurta Set with Jacket?',
  'answer': 'The primary color of the kurta set is beige.'},
 {'query': 'What is the product brand of the jeans described in the document?',
  'answer': 'The product brand of the jeans described in the document is SPYKAR.'},
 {'query': 'What is the product name of the suit mentioned in the document?',
  'answer': 'The product name of the suit mentioned in the document is "Raymond Men Blue Self-Design Single-Breasted Bandhgala

In [36]:
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'No, the Kenneth Cole Women Navy Blue Solid Backpack is not water-resistant.'

In [40]:
import langchain
langchain.debug = False

In [38]:
qa.run(examples[0]["query"])

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "IS Kenneth Cole Women Navy Blue Solid Backpack PaddedWater-resistance?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "IS Kenneth Cole Women Navy Blue Solid Backpack PaddedWater-resistance?",
  "context": ": 10\nProductID: 10000911\nProductName: Kenneth Cole Women Navy Blue Solid Backpack\nProductBrand: Kenneth Cole\nGender: Women\nPrice (INR): 2463\nNumImages: 5\nDescription: Navy Blue backpackNon-Padded haul loop1 main compartment with zip closurePadded backZip PocketPadded shoulder strap: PaddedWater-resistance: No\nPrimaryColor: Blue<<<<>>>>>: 9\nProductID: 10017421\nProductName: DKNY Unisex Black Large Trolley Bag\nProductBrand: DKNY\nGender: Unisex\nPrice (INR): 17360\nNumImages: 5\nDescription: Black solid la

'No, the Kenneth Cole Women Navy Blue Solid Backpack is not water-resistant.'

In [41]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [42]:
from langchain.evaluation.qa import QAEvalChain

In [44]:
eval_chain = QAEvalChain.from_llm(ChatOpenAI(temperature=0))

In [45]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [47]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [48]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: IS Kenneth Cole Women Navy Blue Solid Backpack PaddedWater-resistance?
Real Answer: no
Predicted Answer: No, the Kenneth Cole Women Navy Blue Solid Backpack is not water-resistant.
Predicted Grade: CORRECT

Example 1:
Question: Which product in the Parx brand has a polo collar
Real Answer: Parx Men Green Printed Polo Collar T-shirt
Predicted Answer: The product with a polo collar in the Parx brand is the "Parx Men Green Printed Polo Collar T-shirt" with ProductID 10000245.
Predicted Grade: CORRECT

Example 2:
Question: What is the product name and brand mentioned in the document?
Real Answer: The product mentioned in the document is the "DKNY Unisex Black & Grey Printed Medium Trolley Bag" and the brand is "DKNY".
Predicted Answer: Product 1:
Product Name: Parx Men Brown & Off-White Slim Fit Printed Casual Shirt
Product Brand: Parx

Product 2:
Product Name: Parx Men Green Printed Polo Collar T-shirt
Product Brand: Parx

Product 3:
Product Name: ID Men Beige Leather